In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
#print(g_key)

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Cape Town,ZA,2022-02-05 22:51:37,-33.9258,18.4232,72.10,64,0,1.01
1,1,Saint-Philippe,RE,2022-02-05 22:52:44,-21.3585,55.7679,80.19,83,100,17.74
2,2,Mahebourg,MU,2022-02-05 22:52:41,-20.4081,57.7000,79.25,89,40,11.50
3,3,Kieta,PG,2022-02-05 22:54:49,-6.2167,155.6333,85.15,70,77,8.25
4,4,Richards Bay,ZA,2022-02-05 22:51:59,-28.7830,32.0377,71.31,88,98,7.23


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [11]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],
                                dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Saint-Philippe,RE,2022-02-05 22:52:44,-21.3585,55.7679,80.19,83,100,17.74
2,2,Mahebourg,MU,2022-02-05 22:52:41,-20.4081,57.7000,79.25,89,40,11.50
3,3,Kieta,PG,2022-02-05 22:54:49,-6.2167,155.6333,85.15,70,77,8.25
6,6,Kavieng,PG,2022-02-05 22:46:20,-2.5744,150.7967,82.98,73,63,5.03
14,14,Atuona,PF,2022-02-05 22:54:06,-9.8000,-139.0333,78.80,78,10,22.50
16,16,Hilo,US,2022-02-05 22:50:17,19.7297,-155.0900,76.24,81,100,5.75
18,18,Rikitea,PF,2022-02-05 22:50:20,-23.1203,-134.9692,79.20,73,97,10.67
20,20,Hithadhoo,MV,2022-02-05 22:50:21,-0.6000,73.0833,81.77,74,72,10.69
22,22,Avarua,CK,2022-02-05 22:51:43,-21.2078,-159.7750,80.65,89,40,17.27
30,30,Port-Gentil,GA,2022-02-05 22:54:56,-0.7193,8.7815,80.56,83,40,4.61


In [16]:
preferred_cities_df.count()

City_ID       129
City          129
Country       129
Date          129
Lat           129
Lng           129
Max Temp      129
Humidity      129
Cloudiness    129
Wind Speed    129
dtype: int64

In [17]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Saint-Philippe,RE,80.19,-21.3585,55.7679,
2,Mahebourg,MU,79.25,-20.4081,57.7000,
3,Kieta,PG,85.15,-6.2167,155.6333,
6,Kavieng,PG,82.98,-2.5744,150.7967,
14,Atuona,PF,78.80,-9.8000,-139.0333,
16,Hilo,US,76.24,19.7297,-155.0900,
18,Rikitea,PF,79.20,-23.1203,-134.9692,
20,Hithadhoo,MV,81.77,-0.6000,73.0833,
22,Avarua,CK,80.65,-21.2078,-159.7750,
30,Port-Gentil,GA,80.56,-0.7193,8.7815,


In [34]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   # Grab the first hotel from the results and store the name.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [35]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Saint-Philippe,RE,80.19,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
2,Mahebourg,MU,79.25,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
3,Kieta,PG,85.15,-6.2167,155.6333,Uruna Bay Retreat
6,Kavieng,PG,82.98,-2.5744,150.7967,Nusa Island Retreat
14,Atuona,PF,78.80,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...
525,Vao,NC,80.19,-22.6667,167.4833,Hotel Kou-Bugny
526,Calabozo,VE,89.44,8.9242,-67.4293,Hotel Giardini
527,Tecoanapa,MX,82.85,16.5167,-98.7500,Hotel Manglares
528,Daru,PG,80.42,-9.0763,143.2092,Daru Lodge


In [36]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [40]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [41]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))